Datos de los taxis amarillos de New York City 

In [2]:
import pandas as pd
from pathlib import Path

Cargo un arhivo .parquet para ver su composición 

In [3]:
df_taxiamarillo=pd.read_parquet('yellow_tripdata_2023-04.parquet')

In [4]:
df_taxiamarillo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3288250 entries, 0 to 3288249
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

Creo una referencia de los atributos de cada columna para solo procesar las columnas relevantes para el proyecto

In [ ]:
VendorID: Identificador del proveedor de servicio de taxi.
tpep_pickup_datetime: Fecha y hora de recogida del pasajero.
tpep_dropoff_datetime: Fecha y hora de finalización del viaje.
passenger_count: Cantidad de pasajeros en el viaje.
trip_distance: Distancia recorrida en el viaje.
RatecodeID: Identificador del código de tarifa utilizado para el viaje.
store_and_fwd_flag: Indicador de si el vehículo almacenó los datos del viaje antes de enviarlos.
PULocationID: Identificador de la ubicación de recogida.
DOLocationID: Identificador de la ubicación de destino.
payment_type: Tipo de pago utilizado en el viaje.
fare_amount: Tarifa básica del viaje.
extra: Cargo adicional en la tarifa.
mta_tax: Impuesto MTA en la tarifa.
tip_amount: Cantidad de propina.
tolls_amount: Monto de peajes.
improvement_surcharge: Cargo de mejora en la tarifa.
total_amount: Monto total pagado por el pasajero.
congestion_surcharge: Cargo por congestión en la tarifa.
airport_fee: (tarifa o cargo adicional que se cobra a los pasajeros cuando un viaje involucra un trayecto hacia o desde un aeropuerto).

Defino la carpeta donde se encuntran los archivos .parquet

In [21]:
folder_path = 'C:\\Users\\yopab\\Downloads\\Datasets\\dfexternos\\taxi amarillo'

 Obtengo la lista de archivos .parquet en la carpeta

In [22]:
parquet_files = Path(folder_path).glob('*.parquet')

Filtro los registros que correspondan al año 2021

In [6]:
parquet_files_2021 = [file for file in parquet_files if '2021' in file.name]

Inicializo una lista para almacenar los df de cada archivo

In [10]:
data_frames_2021 = []

Cargo y combino los df 

In [12]:
for file in parquet_files_2021:
    df = pd.read_parquet(file)
    data_frames_2021.append(df)

Combino los df en uno solo

In [13]:
combined_df_2021 = pd.concat(data_frames_2021, ignore_index=True)

Convierto las columnas de fechas a datetime

In [19]:
combined_df_2021['tpep_pickup_datetime'] = pd.to_datetime(combined_df_2021['tpep_pickup_datetime'])
combined_df_2021['tpep_dropoff_datetime'] = pd.to_datetime(combined_df_2021['tpep_dropoff_datetime'])

Agrego una columna para el día

In [20]:
combined_df_2021['pickup_date'] = combined_df_2021['tpep_pickup_datetime'].dt.date

Agrupo por día y sumo los valores diarios que me sirven para el modelo

In [22]:
daily_totals = combined_df_2021.groupby('pickup_date').agg({
    'trip_distance': 'sum',
    'fare_amount': 'sum',
    'tip_amount': 'sum',
    'PULocationID': 'sum',# Identificador de la ubicación de recogida.
    'DOLocationID': 'sum',#Identificador de la ubicación de destino.
    'total_amount': 'sum',#Monto total pagado por el pasajero.
    'payment_type': 'sum', #Tipo de pago utilizado en el viaje.

}).reset_index()

Guardo el el df como archivo CSV

In [23]:
daily_totals.to_csv('totales_diarios_tamarillo.csv', index=False)

compruebo que todo haya salido bien 

In [24]:
totalesAmarillo=pd.read_csv('totales_diarios_tamarillo.csv')

In [27]:
totalesAmarillo

,pickup_date,trip_distance,fare_amount,tip_amount,PULocationID,DOLocationID,total_amount,payment_type
0,2002-12-31,0.00,0.0,0.00,193,193,0.00,1
1,2003-01-01,16.53,57.5,4.00,681,486,76.70,7
2,2003-01-03,0.00,0.0,0.00,193,193,0.00,1
3,2003-01-05,0.00,0.0,0.00,457,457,0.00,2
4,2003-01-09,0.00,2.5,0.01,193,193,4.31,1
...,...,...,...,...,...,...,...,...
394,2022-03-14,2.52,12.0,4.11,381,475,22.71,2
395,2028-12-07,16.15,52.0,0.00,132,170,63.10,2
396,2029-05-05,1.69,7.5,2.16,231,249,12.96,1
397,2070-08-29,0.00,0.0,0.00,193,193,0.00,1


El mismo proceso se repite con el año 2022 y 2023, opteniendo los df separdos 

In [4]:
parquet_files_2022 = [file for file in parquet_files if '2022' in file.name]

In [5]:
data_frames_2022 = []

In [6]:
for file in parquet_files_2022:
    df = pd.read_parquet(file)
    data_frames_2022.append(df)

In [7]:
combined_df_2022 = pd.concat(data_frames_2022, ignore_index=True)

In [9]:
combined_df_2022['tpep_pickup_datetime'] = pd.to_datetime(combined_df_2022['tpep_pickup_datetime'])
combined_df_2022['tpep_dropoff_datetime'] = pd.to_datetime(combined_df_2022['tpep_dropoff_datetime'])

In [11]:
combined_df_2022['pickup_date'] = combined_df_2022['tpep_pickup_datetime'].dt.date

In [12]:
daily_totals = combined_df_2022.groupby('pickup_date').agg({
    'trip_distance': 'sum',
    'fare_amount': 'sum',
    'tip_amount': 'sum',
    'PULocationID': 'sum',# Identificador de la ubicación de recogida.
    'DOLocationID': 'sum',#Identificador de la ubicación de destino.
    'total_amount': 'sum',#Monto total pagado por el pasajero.
    'payment_type': 'sum', #Tipo de pago utilizado en el viaje.

}).reset_index()

In [13]:
daily_totals.to_csv('totales_diarios_tamarillo22.csv', index=False)

In [14]:
totales_diarios_tamarillo22=pd.read_csv('totales_diarios_tamarillo22.csv')

In [15]:
totales_diarios_tamarillo22

,pickup_date,trip_distance,fare_amount,tip_amount,PULocationID,DOLocationID,total_amount,payment_type
0,2001-01-01,36.38,114.50,3.50,878,1102,145.05,8
1,2001-08-23,7.08,22.50,0.00,132,76,24.55,2
2,2002-10-21,215.36,994.00,57.99,6914,6972,1197.49,67
3,2002-10-22,185.00,1619.16,90.97,11906,11744,1972.73,120
4,2002-10-23,253.59,1198.90,109.88,10748,10060,1556.28,104
...,...,...,...,...,...,...,...,...
379,2022-12-29,354466.39,1924170.87,320613.95,15227634,15017348,2761350.25,119677
380,2022-12-30,363679.62,1919571.01,319977.54,14909759,14658450,2757787.48,117871
381,2022-12-31,467628.58,1600107.80,269010.57,13706732,13429150,2272815.42,105911
382,2023-01-01,8.76,75.80,13.90,686,702,111.20,6


In [23]:
parquet_files_2023 = [file for file in parquet_files if '2023' in file.name]

In [24]:
data_frames_2023 = []

In [25]:
for file in parquet_files_2023:
    df = pd.read_parquet(file)
    data_frames_2023.append(df)

In [26]:
combined_df_2023 = pd.concat(data_frames_2023, ignore_index=True)

In [27]:
combined_df_2023['tpep_pickup_datetime'] = pd.to_datetime(combined_df_2023['tpep_pickup_datetime'])
combined_df_2023['tpep_dropoff_datetime'] = pd.to_datetime(combined_df_2023['tpep_dropoff_datetime'])

In [28]:
combined_df_2023['pickup_date'] = combined_df_2023['tpep_pickup_datetime'].dt.date

In [29]:
daily_totals = combined_df_2023.groupby('pickup_date').agg({
    'trip_distance': 'sum',
    'fare_amount': 'sum',
    'tip_amount': 'sum',
    'PULocationID': 'sum',# Identificador de la ubicación de recogida.
    'DOLocationID': 'sum',#Identificador de la ubicación de destino.
    'total_amount': 'sum',#Monto total pagado por el pasajero.
    'payment_type': 'sum', #Tipo de pago utilizado en el viaje.

}).reset_index()

In [30]:
daily_totals.to_csv('totales_diarios_tamarillo23.csv', index=False)

In [31]:
totales_diarios_tamarillo23=pd.read_csv('totales_diarios_tamarillo23.csv')

In [32]:
totales_diarios_tamarillo23

,pickup_date,trip_distance,fare_amount,tip_amount,PULocationID,DOLocationID,total_amount,payment_type
0,2001-01-01,64.76,273.1,23.46,1149,946,337.66,9
1,2002-12-31,28.51,125.2,0.00,642,348,148.75,8
2,2003-01-01,76.48,368.2,22.12,789,1061,428.70,9
3,2008-12-31,102.99,430.9,56.96,1853,2143,576.11,20
4,2009-01-01,1.92,14.4,0.00,293,406,20.90,4
...,...,...,...,...,...,...,...,...
166,2023-07-05,6.63,48.6,17.01,841,968,91.11,5
167,2023-07-25,1.29,8.6,2.72,230,68,16.32,1
168,2023-07-27,4.25,30.5,9.11,118,304,50.11,2
169,2023-08-15,6.00,33.3,8.06,136,511,51.36,2
